# 0. Import required packages.

In [1]:
import copy
import time

# 1. Define a `Game` Class for containing the Tic-Tac-Toe game.

In [2]:
class Game(object):
    """A tic-tac-toe game."""

    def __init__(self, grid):
        """Instances differ by their grid marks."""
        self.grid = copy.deepcopy(grid) # No aliasing!

    def display(self):
        """Print the game board."""
        for row in self.grid:
            print(row)

    def moves(self):
        """Return a list of possible moves given the current marks.
        
        Parameters
        ----------
            None
        
        Returns
        -------
            possible_moves: List of Tuples containing (row, column) of available spaces on the grid.
        
        Author: Miguel Agueda-Cabral
        """
        
        possible_moves = []
        for row_index, row in enumerate(self.grid):
            for column_index, value in enumerate(row):
                if value == '-':
                    position = (row_index, column_index)
                    possible_moves.append(position)
        
        return possible_moves

    def neighbor(self, move, mark):
        """Return a Game instance like this one but with one move made.
        
        Parameters
        ----------
            move: Tuple containing (row, column) indices of move to be made.
            mark: String identity of player ('X', 'O'). 
        
        Returns
        -------
            neighbor_game: A copy of `self` with `move` applied.
        
        Author: Miguel Agueda-Cabral
        """
        
        neighbor_game = Game(self.grid)
        (row_idx, col_idx) = move
        neighbor_game.grid[row_idx][col_idx] = mark
        
        return neighbor_game

    def utility(self):
        """Return the minimax utility value of this game:
        1 = X win, -1 = O win, 0 = tie, None = not over yet."""
        # YOU FILL THIS IN
        # for utility here, check the code from this link
        #http://www.sarathlakshman.com/2011/04/29/writing-a-tic-tac
        #to calculate the utility of this current board.

   # 2. Define an `Agent` Class which can manipulate and solve an instance of `Game`.

In [3]:
class Agent(object):
    """Knows how to play tic-tac-toe."""

    def __init__(self, mark):
        """Agents use either X or O."""
        self.mark = mark

    def maxvalue(self, game, opponent):
        """Compute the highest utility this game can have."""
        # YOU FILL THIS IN

    def minvalue(self, game, opponent):
        """Compute the lowest utility this game can have."""
        # YOU FILL THIS IN



In [4]:
def main():
    """Create a game and have two agents play it."""

    game = Game([['-','-','-'], ['-','-','-'], ['-','-','-']])
    game.display()

    maxplayer = Agent('X')
    minplayer = Agent('O')

    while True:

        (value, move) = maxplayer.maxvalue(game, minplayer)
        game = game.neighbor(move, maxplayer.mark)
        time.sleep(1)
        game.display()
        
        if game.utility() is not None:
            break
        
        (value, move) = minplayer.minvalue(game, maxplayer)
        game = game.neighbor(move, minplayer.mark)
        time.sleep(1)
        game.display()
        
        if game.utility() is not None:
            break

if __name__ == '__main__':
    main()

['-', '-', '-']
['-', '-', '-']
['-', '-', '-']


TypeError: cannot unpack non-iterable NoneType object